In [81]:
import networkx as nx
import random
from networkx.algorithms.flow import edmonds_karp

class flow:
    def GenerateGraph(self):
        """
        Generrate a connected,indegeed,outdegreed,forad edged graph
        Returns:
            gaph object
        """
        try:
            flag = 0
            while flag == 0:
                self.G = nx.gnp_random_graph(30, 0.1, directed=True)
                connect = nx.is_weakly_connected(self.G)
                if not connect:
                    continue

                for u, v in self.G.edges():
                    capacity = random.randint(1, 10)
                    flow = random.randint(1, capacity)
                    self.G[u][v]['flow'] = flow
                    self.G[u][v]['capacity'] = capacity
                if not self.check_forward_edges_exist():
                    continue
                if not self.check_incoming_edges():
                    continue
                if not self.check_outgoing_edges():
                    continue

                flag = 1

            return self.G
        except Exception as e:
            raise e

    def check_forward_edges_exist(self):
        """
        Checks if forward edge exists or not
        Returns:
            return true or false
        """
        for u, v in self.G.edges():
            if self.G[u][v]['flow'] > self.G[u][v]['capacity']:
                return False
        return True

    def check_incoming_edges(self):
        """
        Checks if a graph has atleast one incoming node apart from strating node
        Returns:
            true or false
        """
        for node in range(1, 30):  
            if not self.G.in_degree(node):
                return False
        return True

    def check_outgoing_edges(self):
        """
        Checks if a graph has atleast one incoming node apart from strating node
        Returns:
            true or false
        """
        for node in range(0, 29):  
            if not self.G.out_degree(node):
                return False
        return True   
    def Paths_From_Source_to_Destination(self,Source,Destination):
        """
        It will give the possible valid paths between source to destination
        Args:
        G:Graph object
        Source:Source node
        Destination:Destination node
        Result
        Returns a list of valid paths between source and destination
        """
        try:
            paths=[]                                  
            for path in nx.all_simple_edge_paths(self.G, source=Source, target=Destination, cutoff=9):
                flag=0     
                for u, v in path:
                    dir=self.G.get_edge_data(u,v)
                    if dir['flow']==dir['capacity']:
                        flag=1
                        break
                if flag==0:
                    paths.append(path)
            return paths
        except Exception as e:
            raise e
    def Heuristic(self,path,G):
        """
        Takes  path between source and destination returns minimmum heuristic value
        Arg:
        paths:Valid path between source and destination
        G:Graph object
        Returns:Returns minimum heuristic value in a paths
        """
        try:
            minimum_heuristic=30
            for i in path:
                u=i[0]
                v=i[1]
                dir=self.G.get_edge_data(u,v)
                capacity=dir['capacity']
                flow=dir['flow']
                minimum_heuristic=min(minimum_heuristic,capacity-flow)
            return minimum_heuristic
        except Exception as e:
            raise e
    def BestHeuristic(self,heuristic_path_values):
        """
        It takes heuristic values of every path as argument and return best path 
        Args:
        Heuristic_Best_Path:It is a dictionary which has path as key and its heuristic value as value
        """            
        try:
            sum=0
            for i in heuristic_path_values:
                len1=len(i)
                sum=sum+i[len1-1]
            best_path=[]
            best_value=0
            best_valve=0
            for i in heuristic_path_values:
                len1=len(i)
                probability=i[len1-1]/sum
                if probability>best_value:
                    best_path=i
                    best_value=probability
            return best_path
        except Exception as e:
            raise e
    
    def Best_Path(self,valid_paths):
        """
        It takes valid paths as argument and return best path
        Args:
        valid_paths:It's a list containing valid paths that connect Source to Destination 
        """              
        try:
            heuristic_path_values=[]
            for path in valid_paths:
                heuristic_value=self.Heuristic(path,self.G)
                list1=path
                path.append(heuristic_value)
                heuristic_path_values.append(path)
            best_path=self.BestHeuristic(heuristic_path_values)
            return best_path
        except Exception as e:
            raise e
    
    def Increase_Flow(self,best_path):
        """
        It takes a best path dictionary as argument and updates the graph by increase the flow of every edge in the best path
        Args:
        best_path:It contain best path and heuristic value as key and value pair
        """    
        try:
            len1=len(best_path)
            value=best_path[len1-1]
            c=best_path.pop(len1-1)
            path=best_path
            for i in path:
                u=i[0]
                v=i[1]
                self.G.get_edge_data(u,v)['flow']=self.G.get_edge_data(u,v)['flow']+value
            return self.G
        except Exception as e:
            raise e
    
    def Successor(self,Source,Destination):
        """
        It takes Source,Destination as arguments and return valid path
        Args:
        Source:It's the starting point of flow
        Destination:It's the ending point point of flow
        """
        try:
            valid_paths=self.Paths_From_Source_to_Destination(Source,Destination)
            return valid_paths
        except Exception as e:
            raise e
    def in_flow(self,edges):
        """
        Takes edges as argument and return inflow of the output
        Args:
            edges:list 
        Returns:
            sum:returns inflow sum
        """
        try:
            sum=0
            print(edges)
            for i in edges:
                u=i[0]
                v=i[1]
                sum=sum+self.G.get_edge_data(u,v)['flow']
            return sum
        except Exception as e:
            raise e    
    def predict_flow(self):
        """
        This functions gives the optimum flow for a given graph
        returns:
        optimum inflow of target node 
        """
        try:
            R = edmonds_karp(self.G, 0, 29)
            flow_value = nx.maximum_flow_value(R, 0, 29)   
            return flow_value
        except Exception as e:
            raise e                     
        
    def __init__(self):
        self.GenerateGraph()
        heuristic_value = 30
        predicted = self.predict_flow()
        edges = list(self.G.in_edges(29))
        sum1 = self.in_flow(edges)
        while heuristic_value != 0:
            valid_paths = self.Successor(0, 29)
            best_path = self.Best_Path(valid_paths)
            print(best_path)
            len1 = len(best_path)
            if len1 > 0:
                heuristic_value = best_path[len1 - 1]
                self.Increase_Flow(best_path)
            else:
                heuristic_value = 0
        #edges = list(self.G.in_edges(29))
        sum2 = self.in_flow(edges)
        print("inflow before modification", sum1)
        print("inflow after modification tf=", sum2)
        print("predicted optimum flow value by Networkx tf_net=", predicted)
        
                
re=flow()            
                                  
                                   



[(2, 29), (16, 29)]
[(0, 6), (6, 22), (22, 1), (1, 23), (23, 28), (28, 3), (3, 10), (10, 16), (16, 29), 1]
[(0, 26), (26, 10), (10, 16), (16, 29), 1]
[]
[(2, 29), (16, 29)]
inflow before modification 5
inflow after modification tf= 7
predicted optimum flow value by Networkx tf_net= 8
